In [3]:
import os

from pyspark.sql import SparkSession, functions as F
import logging

from utils import read_geo, read_events, add_closest_city

In [4]:
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["YARN_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"

In [5]:
# DEV Constants
GEO_DIR = "/user/solovyovyu/geo.csv"
EVENTS_DIR = "/user/solovyovyu/data/geo/events"
OUT_PATH = "/user/solovyovyu/analytics"

# PROD Constants
# EVENTS_DIR = "/user/master/data/geo/events"

In [6]:
def setup_logging():
    """Configure logging"""
    logger = logging.getLogger(__name__)
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
    return logger

logger = setup_logging()

In [7]:
try:
    spark = SparkSession.builder \
        .master("local") \
        .appName("Mart User") \
        .getOrCreate()
    logger.info("SparkSession создан успешно.")
except Exception as e:
    logger.error(f"Ошибка при создании SparkSession: {e}")
    raise

your 131072x1 screen size is bogus. expect trouble
25/02/12 17:38:25 WARN Utils: Your hostname, DESKTOP-BBBONO9 resolves to a loopback address: 127.0.1.1; using 172.20.57.228 instead (on interface eth0)
25/02/12 17:38:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/12 17:38:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2025-02-12 17:38:30,763 - INFO - SparkSession создан успешно.


In [8]:
geo_df = read_geo(GEO_DIR, spark, logger)

2025-02-12 17:38:36,431 - INFO - Geo data is read from /user/solovyovyu/geo.csv.


In [9]:
messages_df = read_events("message", EVENTS_DIR, spark, logger)

2025-02-12 17:38:37,217 - INFO - Events message are read from /user/solovyovyu/data/geo/events.


In [10]:
reaction_df = read_events("reaction", EVENTS_DIR, spark, logger)

2025-02-12 17:38:37,486 - INFO - Events reaction are read from /user/solovyovyu/data/geo/events.


In [11]:
subscription_df = read_events("subscription", EVENTS_DIR, spark, logger)

2025-02-12 17:38:37,769 - INFO - Events subscription are read from /user/solovyovyu/data/geo/events.


In [13]:
messages_with_city = add_closest_city(messages_df, geo_df, logger)

messages_with_city.cache()

messages_with_city.show()

25/02/12 17:38:53 WARN CacheManager: Asked to cache already cached data.


PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/samogonn/projects/de-project-sprint-7/src/scripts/utils.py", line 3, in <module>
    from timezonefinder import TimezoneFinder
ModuleNotFoundError: No module named 'timezonefinder'


In [11]:
messages_with_weeks = messages_with_city \
    .withColumn("event_date", F.to_date(F.col("datetime"))) \
    .withColumn("year", F.year("event_date")) \
    .withColumn("month", F.month("event_date")) \
    .withColumn("week", F.weekofyear("event_date"))

In [22]:
messges_by_weeks = messages_with_weeks.groupBy("year", "week", "city") \
    .agg(
        F.count("message_id").alias("week_message")
    )

In [23]:
messges_by_month = messages_with_weeks \
    .groupBy("year", "month", "city") \
    .agg(
        F.count("message_id").alias("month_message")
    )

In [24]:
subscription_with_city = add_closest_city(subscription_df, geo_df, logger)

subscription_with_city.show()

TypeError: add_closest_city() takes 2 positional arguments but 3 were given